#AIS WORKSHOP 2
### BUILDING A COOL APPLICATION USING BERT

## Dataset
Dataset consists of celebrity tweets, the name of the celebrity who tweeted it and the sentiment of the tweet.
Yu can find the dataset here.
https://github.com/estorrs/twitter-celebrity-tweet-sentiment/blob/master/results/celebrity_tweets_results.csv

## Models: Sentence Sentiment Classification
Our goal is to create a model that takes a tweet (just like the ones in our dataset) and produces name of the celebrity that tweeted it. 

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and identify the celebrity who must have tweeted it.



## Installing the transformers library
Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [0]:
!pip install -q transformers

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/estorrs/twitter-celebrity-tweet-sentiment/master/results/celebrity_tweets_results.csv', header=None)
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
df[0]

In [0]:
df.head(5)

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [0]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [0]:
tokenized = df[1].apply(lambda x: tokenizer.encode(str(x), add_special_tokens=True))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [0]:
np.array(padded).shape

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [0]:
example = df[1][53]
example

In [0]:
example.split()

In [0]:
tokenized.values[53]

In [0]:
padded[53]

In [0]:
attention_mask[53]

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [0]:
input_ids = torch.tensor(padded[:300])  
attention_mask = torch.tensor(attention_mask[:300])

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [0]:
labels = df[0][:300]

In [0]:
labels.shape

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,442 sentences from the training set).

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, shuffle=False)

In [0]:
train_labels

In [0]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

<img src="http://127.0.0.1:4000/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [0]:
lr_clf.score(test_features, test_labels)



## Improve Scores

And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.

## Quiz the Audience

Let's play a game and test our `model`, and audience, with which celebrity said which quote.



Here's are the celebrities in the dataset to choose from.

In [0]:
df[0].unique()

In [0]:
random_number = 70 # Put your own! Within bounds 0 to 814
df[1][df.shape[0] - test_features.shape[0] + random_number]

In [0]:
lr_clf.predict(test_features[random_number, np.newaxis])

In [0]:
test_labels.iloc[random_number]